In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy as sp
from textblob import TextBlob
import pandas as pd
import numpy as np


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nlplab/harry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# read training data
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [3]:
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def tokenizer(text):
    return re.split('\s+', text.strip())

def tokenizer_stem(text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in re.split('\s+', text.strip())]

def tokenizer_stem_nostop(text):
    stop = stopwords.words('english')
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk

In [4]:
def fetch_datetime(soup):
    if soup.time.has_attr('datetime'):
        date = soup.time.attrs['datetime']
        day = ' '+ date[0:3]
    else:
        day  = ' noneday'
    return day

def fetch_pubday(soup):
    if soup.time.has_attr('datetime'):
        date = soup.time.attrs['datetime']
        pub_day = '' + date[:]
    else:
        pub_day =  ' noneday'
    return pub_day

def fetch_channel(soup):
    
    channel = soup.article['data-channel']
    return channel

def fetch_img_count(soup):

    c = 0
    find_all_images = soup.find_all('img')

    for i in find_all_images:
        c = c+1
    return c

def fetch_topics(soup):

    footer = soup.footer
    ta = footer.find_all('a')
    topic = []

    for t in ta:
        topic.append(t.get_text())
    topic_text = ' '.join(topic)

    return topic_text

def fetch_authors(soup):

    footer = soup.span
    if footer != None:
        ta = footer.findAll('a')
        authors = []
        for t in ta:
            authors.append(t.get_text())
        if len(authors) == 0:
            authors_text = 'NaN'
        else:
            authors_text = ''.join(authors)
    else:
        authors_text   = 'NaN'

    return authors_text

def fetch_titles(soup):
    
    footer = soup.h1
    if footer != None:
        titles = footer.get_text()
    else:
        titles = 'NaN'
    return titles

def fetch_social_media_count(soup):

    c = 0
    for frame in soup("iframe"):
        if frame.get('src').find("youtube") != None:
            c = c+1
        elif frame.get('src').find("instagram") != None:
            c = c+1
        elif frame.get('src').find("vine") != None:
            c = c+1

    return c


def fetch_href(soup):
    all_a_tags = soup.find_all('a', href=True)
    num_href = len(all_a_tags)
    num_self_href = 0
    for tag in all_a_tags:
        href = tag['href']
        if 'mashable' in href:
            num_self_href += 1
    return num_href, num_self_href



In [5]:
df_train_contents = df_train['Page content'].values.tolist()

In [6]:
def get_all_datas(texts):
    days = []
    channels = []
    img_counts = []
    topics = []
    authors = []
    titles = []
    social_media_counts = []
    contents = []
    num_hrefs = []
    num_self_hrefs = []
    pub_days = []
    
    for text in texts:
        soup = BeautifulSoup(text, "lxml")
        contents.append(soup.find('article').get_text())
        topics.append(fetch_topics(soup))
        channels.append(fetch_channel(soup))
        days.append(fetch_datetime(soup))
        authors.append(fetch_authors(soup))
        img_counts.append(fetch_img_count(soup))
        titles.append(fetch_titles(soup))
        social_media_counts.append(fetch_social_media_count(soup))
#         input()
        
        num_href, num_self_href = fetch_href(soup)
        num_hrefs.append(num_href)
        num_self_hrefs.append(num_self_href)
        pub_days.append(fetch_pubday(soup))
        

    return days, pub_days, channels, img_counts, topics, authors, titles, social_media_counts, contents, num_hrefs, num_self_hrefs

In [7]:
days, pub_days,  channels, img_counts, topics, authors, titles, social_media_counts, contents, num_hrefs, num_self_hrefs = get_all_datas(df_train_contents)

In [8]:
print(len(days))
print(len(pub_days))
print(len(channels))
print(len(img_counts))
print(len(topics))
print(len(authors))
print(len(titles))
print(len(social_media_counts))
print(len(contents))
print(len(num_hrefs))
print(len(num_self_hrefs))

27643
27643
27643
27643
27643
27643
27643
27643
27643
27643
27643


In [9]:
# 2. n_tokens_title: Number of words in the title
# 3. n_tokens_content: Number of words in the content
# 4. n_unique_tokens: Rate of unique words in the content
# 5. n_non_stop_words: Rate of non-stop words in the content
# 6. n_non_stop_unique_tokens: Rate of unique non-stop words in the content
import string
porter = PorterStemmer()
from nltk.corpus import stopwords
stop = stopwords.words('english')
def process(text):
    
    # Remove HTML tags.
#     text = BeautifulSoup(text,'html.parser').get_text()
#     text = re.sub("[^a-zA-Z]", "", text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    
    
    tokens  = nltk.word_tokenize(text)
    tokens = [porter.stem(w) for w in tokens]

#     token = text.split()
#     text = [porter.stem(w) for w in token if w not in stop]
    # Join the words back into one string separated by space, and return the result.
    return tokens

In [10]:
def get_some_n_features(titles, contents):
    n_tokens_titles = []
    n_tokens_contents = []
    n_unique_tokens = []
    n_non_stop_words = []
    n_non_stop_unique_tokens = []


    for title, content in zip(titles, contents):

        title_tokens = process(title)
        n_tokens_titles.append(len(title_tokens))

        content_tokens = process(content)
        n_tokens_contents.append(len(content_tokens))
        
        len_content_tokens = len(content_tokens)
        if len(content_tokens) == 0:
            print(content)
            len_content_tokens = 1
            
        set_content_token = set(content_tokens)
        unique_token_rate = len(set_content_token) / len_content_tokens
        n_unique_tokens.append(unique_token_rate)

        non_stop_words = [w for w in content_tokens if w not in stop]

        non_stop_word_rate = len(non_stop_words) / len_content_tokens
        n_non_stop_words.append(non_stop_word_rate)

        set_non_stop_words = set(non_stop_words)
        n_non_stop_unique_tokens_rate = len(set_non_stop_words) / len_content_tokens
        n_non_stop_unique_tokens.append(n_non_stop_unique_tokens_rate)
        
        
#         


    return n_tokens_titles, n_tokens_contents, n_unique_tokens, n_non_stop_words, n_non_stop_unique_tokens
    
#     print(n_tokens_titles)
#     print(n_tokens_contents)
#     print(n_unique_tokens)
#     print(n_non_stop_words)
#     print(n_non_stop_unique_tokens)
    

In [11]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# global_sentiment_polarity: Text sentiment polarity
# global_rate_positive_words: Rate of positive words in the content
# global_rate_negative_words: Rate of negative words in the content
# rate_positive_words: Rate of positive words among non-neutral tokens
# rate_negative_words: Rate of negative words among non-neutral tokens
# title_subjectivity: Title subjectivity
# title_sentiment_polarity: Title polarity
# abs_title_subjectivity: Absolute subjectivity level
# abs_title_sentiment_polarity: Absolute polarity level

def get_sentiment_features(titles, contents):
    global_sentiment_polarity = []
    global_subjectivity = []
    
#     global_rate_positive_words = []
#     global_rate_negative_words = []
    
#     rate_positive_words = []
#     rate_negative_words = []
    
    title_subjectivity_list = []
    title_sentiment_polarity_list = []
    
    abs_title_subjectivity = []
    abs_title_sentiment_polarity = []
    
    for title, content in zip(titles, contents):
        title_blob = TextBlob(title)
        title_polarity = title_blob.sentiment.polarity
        title_subjectivity = title_blob.sentiment.subjectivity

        
        title_sentiment_polarity_list.append(title_polarity)
        title_subjectivity_list.append(title_subjectivity)
        abs_title_subjectivity.append(abs(title_subjectivity))
        abs_title_sentiment_polarity.append(abs(title_polarity))
        
        
        content_blob = TextBlob(content)
        content_polarity = content_blob.sentiment.polarity
        content_subjectivity = content_blob.sentiment.subjectivity
        global_sentiment_polarity.append(content_polarity)
        global_subjectivity.append(content_subjectivity)
        
        
        
        
    return global_sentiment_polarity, global_subjectivity, title_subjectivity_list, title_sentiment_polarity_list, \
abs_title_subjectivity, abs_title_sentiment_polarity

        
        

In [12]:
# global_rate_positive_words: Rate of positive words in the content
# global_rate_negative_words: Rate of negative words in the content
# rate_positive_words: Rate of positive words among non-neutral tokens
# rate_negative_words: Rate of negative words among non-neutral tokens
# avg_positive_polarity: Avg. polarity of positive words
# min_positive_polarity: Min. polarity of positive words
# max_positive_polarity: Max. polarity of positive words
# avg_negative_polarity: Avg. polarity of negative words
# min_negative_polarity: Min. polarity of negative words
# max_negative_polarity: Max. polarity of negative words
def get_word_sentiment_features(contents):
#     global_rate_positive_words = []
#     global_rate_negative_words = []
    
    rate_positive_words = []
    rate_negative_words = []
    avg_positive_polarity = []
    min_positive_polarity = []
    max_positive_polarity = []
    avg_negative_polarity = []
    min_negative_polarity = []
    max_negative_polarity = []
    
    for content in contents:
        content_tokens = process(content)
        
        pos_count = 0
        pos_score = 0.0
        min_pos_polarity = 1.0
        max_pos_polarity = 0.0
        
        neg_count = 0
        neg_score = 0.0
        min_neg_polarity = 0.0
        max_neg_polarity = -1.0
        
        for token in content_tokens:
            blob = TextBlob(token)
            sentiment_score = blob.sentiment.polarity
#             positive
            if sentiment_score > 0.0:
                pos_count += 1
                pos_score += sentiment_score
                
                if sentiment_score < min_pos_polarity:
                    min_pos_polarity = sentiment_score
                if sentiment_score > max_pos_polarity:
                    max_pos_polarity = sentiment_score
#             negative
            elif sentiment_score < 0.0:
                neg_count += 1
                neg_score += sentiment_score
                
                if sentiment_score < min_neg_polarity:
                    min_neg_polarity = sentiment_score
                if sentiment_score > max_neg_polarity:
                    max_neg_polarity = sentiment_score
                    
        if len(content_tokens) == 0:
            rate_positive_words.append(0.0)
            rate_negative_words.append(0.0)
            avg_positive_polarity.append(0.0)
            min_positive_polarity.append(0.0)
            max_positive_polarity.append(0.0)
            avg_negative_polarity.append(0.0)
            min_negative_polarity.append(0.0)
            max_negative_polarity.append(0.0)
            continue
            
        if pos_count == 0:
            max_pos_polarity = 0.0
            min_pos_polarity = 0.0
        else:
            pos_score /= pos_count
            
        if neg_count == 0:
            min_neg_polarity = 0.0
            max_neg_polarity = 0.0
        else:
            neg_score /= neg_count

            
        
        
        pos_rate = pos_count / len(content_tokens)
        neg_rate = neg_count / len(content_tokens)
        rate_positive_words.append(pos_rate)
        rate_negative_words.append(neg_rate)
        
        avg_positive_polarity.append(pos_score)
        min_positive_polarity.append(min_pos_polarity)
        max_positive_polarity.append(max_pos_polarity)
        
        avg_negative_polarity.append(neg_score)
        min_negative_polarity.append(min_neg_polarity)
        max_negative_polarity.append(max_neg_polarity)
        
        
        
    return rate_positive_words, rate_negative_words, avg_positive_polarity, min_positive_polarity, max_positive_polarity,\
avg_negative_polarity, min_negative_polarity, max_negative_polarity

In [13]:
rate_positive_words, rate_negative_words, avg_positive_polarity, min_positive_polarity, max_positive_polarity,\
avg_negative_polarity, min_negative_polarity, max_negative_polarity = get_word_sentiment_features(contents)

print(len(rate_positive_words))
print(len(rate_negative_words))
print(len(avg_positive_polarity))
print(len(min_positive_polarity))
print(len(max_positive_polarity))
print(len(avg_negative_polarity))
print(len(min_negative_polarity))
print(len(max_negative_polarity))

27643
27643
27643
27643
27643
27643
27643
27643


In [14]:
n_tokens_titles, n_tokens_contents, n_unique_tokens, n_non_stop_words, n_non_stop_unique_tokens = get_some_n_features(titles, contents)



In [15]:
print(len(n_tokens_titles))
print(len(n_tokens_contents))
print(len(n_unique_tokens))
print(len(n_non_stop_words))
print(len(n_non_stop_unique_tokens))

27643
27643
27643
27643
27643


In [16]:
global_sentiment_polarity, global_subjectivity, title_subjectivity_list, title_sentiment_polarity_list, \
abs_title_subjectivity, abs_title_sentiment_polarity = get_sentiment_features(titles, contents)

print(len(global_sentiment_polarity))
print(len(global_subjectivity))
print(len(title_subjectivity_list))
print(len(title_sentiment_polarity_list))
print(len(abs_title_subjectivity))
print(len(abs_title_sentiment_polarity))


27643
27643
27643
27643
27643
27643


In [17]:
# re-consturct training data

# 2. n_tokens_title: Number of words in the title
# 3. n_tokens_content: Number of words in the content
# 4. n_unique_tokens: Rate of unique words in the content
# 5. n_non_stop_words: Rate of non-stop words in the content
# 6. n_non_stop_unique_tokens: Rate of unique non-stop words in the content

df_train_feature = pd.DataFrame({'Id':df_train.Id[:],
                           'Popularity':df_train.Popularity[:],
                           'topic':topics,
                           'channel':channels,
                           'weekday':days,
                           'pub_date' : pub_days,
                           'author':authors,
                           'img count':img_counts,
                           'title':titles,
                            'content':contents,
                           'media count': social_media_counts,
                           'n_tokens_title' : n_tokens_titles,
                           'n_tokens_content': n_tokens_contents,
                           'n_unique_tokens' : n_unique_tokens,
                           'n_non_stop_words': n_non_stop_words,
                            'n_non_stop_unique_tokens': n_non_stop_unique_tokens,
                            'num_hrefs' : num_hrefs,
                            'num_self_hrefs' : num_self_hrefs,
                                 'global_sentiment_polarity' : global_sentiment_polarity,
                                 'global_subjectivity' : global_subjectivity,
                                 'title_subjectivity' : title_subjectivity_list,
                                 'title_sentiment_polarity' : title_sentiment_polarity_list,
                                 'abs_title_subjectivity' : abs_title_subjectivity,
                                 'abs_title_sentiment_polarity' : abs_title_sentiment_polarity,
                                 
                                 'rate_positive_words' : rate_positive_words,
                                'rate_negative_words' : rate_negative_words,
                                 'avg_positive_polarity' : avg_positive_polarity,
                                 'min_positive_polarity' : min_positive_polarity,
                                 'max_positive_polarity' : max_positive_polarity,
                                 'avg_negative_polarity' : avg_negative_polarity,
                                 'min_negative_polarity' : min_negative_polarity,
                                 'max_negative_polarity' : max_negative_polarity,

                                })

In [18]:
from time import strptime
import datetime

df_train_feature['day_of_month'] = df_train_feature['pub_date'].apply(lambda x: int(x.split()[1]))
df_train_feature['month'] = df_train_feature['pub_date'].apply(lambda x: strptime(x.split()[2], '%b').tm_mon)
# df_train_feature['day_of_week'] = df_train_feature['pub_date'].apply(lambda x: strptime(x.split()[0][:-1], '%a').tm_wday + 1)
df_train_feature['hour'] = df_train_feature['pub_date'].apply(lambda x: strptime(x.split()[4], '%X')[3])
# df_train_feature['ymd'] = df_train_feature['pub_date'].apply(lambda x: x[5:7] + '-' + x[8:11] + '-' + x[12:16])
#reference: https://www.796t.com/post/aWw5c28=.html
# df_train_feature['ymd'] = pd.to_datetime(df_train_feature['ymd'],  format='%d-%b-%Y')

In [19]:
df_train_feature.head()

,Id,Popularity,topic,channel,weekday,pub_date,author,img count,title,content,...,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,day_of_month,month,hour
0,0,-1,Asteroid Asteroids challenge Earth Space U.S. ...,world,Wed,"Wed, 19 Jun 2013 15:04:30 +0000",NaN,1,NASA's Grand Challenge: Stop Asteroids From De...,There may be killer asteroids headed for Eart...,...,0.012174,0.465702,0.136364,0.8,-0.153571,-0.25,-0.125,19,6,15
1,1,1,Apps and Software Google open source opn pledg...,tech,Thu,"Thu, 28 Mar 2013 17:40:55 +0000",Christina Warren,2,Google's New Open Source Patent Pledge: We Won...,Google took a stand of sorts against patent-l...,...,0.016393,0.386364,0.136364,0.7,-0.130000,-0.25,-0.050,28,3,17
2,2,1,Entertainment NFL NFL Draft Sports Television,entertainment,Wed,"Wed, 07 May 2014 19:15:20 +0000",Sam Laird,2,Ballin': 2014 NFL Draft Picks Get to Choose Th...,You've spend countless hours training to be a...,...,0.042002,0.435118,0.062500,1.0,-0.433992,-1.00,-0.050,7,5,19
3,3,-1,Sports Video Videos Watercooler,watercooler,Fri,"Fri, 11 Oct 2013 02:26:50 +0000",Sam Laird,1,Cameraperson Fails Deliver Slapstick Laughs,Tired of the same old sports fails and ne...,...,0.040146,0.268214,0.100000,0.5,-0.432727,-0.60,-0.150,11,10,2
4,4,-1,Entertainment instagram instagram video NFL Sp...,entertainment,Thu,"Thu, 17 Apr 2014 03:31:43 +0000",Connor Finnegan,52,NFL Star Helps Young Fan Prove Friendship With...,"At 6-foot-5 and 298 pounds, All-Pro NFL star ...",...,0.015328,0.401529,0.062500,1.0,-0.303175,-0.40,-0.050,17,4,3


In [20]:
df_train_feature.to_csv('./train_feature.csv',index=False,header=True)

In [21]:
df_test_contents = df_test['Page content'].values.tolist()
print(len(df_test_contents))
days, pub_days,  channels, img_counts, topics, authors, titles, social_media_counts, contents, num_hrefs, num_self_hrefs = get_all_datas(df_test_contents)
print(len(days))
print(len(channels))
print(len(img_counts))
print(len(topics))
print(len(authors))
print(len(titles))
print(len(social_media_counts))
print(len(contents))
print(len(num_hrefs))
print(len(num_self_hrefs))


11847
11847
11847
11847
11847
11847
11847
11847
11847
11847
11847


In [22]:
n_tokens_titles, n_tokens_contents, n_unique_tokens, n_non_stop_words, n_non_stop_unique_tokens = get_some_n_features(titles, contents)


In [23]:
print(len(n_tokens_titles))
print(len(n_tokens_contents))
print(len(n_unique_tokens))
print(len(n_non_stop_words))
print(len(n_non_stop_unique_tokens))

11847
11847
11847
11847
11847


In [24]:
global_sentiment_polarity, global_subjectivity, title_subjectivity_list, title_sentiment_polarity_list, \
abs_title_subjectivity, abs_title_sentiment_polarity = get_sentiment_features(titles, contents)

print(len(global_sentiment_polarity))
print(len(global_subjectivity))
print(len(title_subjectivity_list))
print(len(title_sentiment_polarity_list))
print(len(abs_title_subjectivity))
print(len(abs_title_sentiment_polarity))

11847
11847
11847
11847
11847
11847


In [25]:
rate_positive_words, rate_negative_words, avg_positive_polarity, min_positive_polarity, max_positive_polarity,\
avg_negative_polarity, min_negative_polarity, max_negative_polarity = get_word_sentiment_features(contents)

print(len(rate_positive_words))
print(len(rate_negative_words))
print(len(avg_positive_polarity))
print(len(min_positive_polarity))
print(len(max_positive_polarity))
print(len(avg_negative_polarity))
print(len(min_negative_polarity))
print(len(max_negative_polarity))

11847
11847
11847
11847
11847
11847
11847
11847


In [26]:
# re-consturct training data
df_test_feature = pd.DataFrame({'Id':df_test.Id[:],
                           'topic':topics,
                           'channel':channels,
                           'weekday':days,
                            'pub_date' : pub_days,
                           'author':authors,
                           'img count':img_counts,
                           'title':titles,
                            'content':contents,
                           'media count': social_media_counts,
                           'n_tokens_title' : n_tokens_titles,
                           'n_tokens_content': n_tokens_contents,
                           'n_unique_tokens' : n_unique_tokens,
                           'n_non_stop_words': n_non_stop_words,
                            'n_non_stop_unique_tokens': n_non_stop_unique_tokens,
                            'num_hrefs' : num_hrefs,
                            'num_self_hrefs' : num_self_hrefs,
                                'global_sentiment_polarity' : global_sentiment_polarity,
                                 'global_subjectivity' : global_subjectivity,
                                 'title_subjectivity' : title_subjectivity_list,
                                 'title_sentiment_polarity' : title_sentiment_polarity_list,
                                 'abs_title_subjectivity' : abs_title_subjectivity,
                                 'abs_title_sentiment_polarity' : abs_title_sentiment_polarity,
                                
                                'rate_positive_words' : rate_positive_words,
                                'rate_negative_words' : rate_negative_words,
                                 'avg_positive_polarity' : avg_positive_polarity,
                                 'min_positive_polarity' : min_positive_polarity,
                                 'max_positive_polarity' : max_positive_polarity,
                                 'avg_negative_polarity' : avg_negative_polarity,
                                 'min_negative_polarity' : min_negative_polarity,
                                 'max_negative_polarity' : max_negative_polarity,
                               })

In [27]:
df_test_feature['day_of_month'] = df_test_feature['pub_date'].apply(lambda x: int(x.split()[1]) if x 
                                                            != ' noneday' else 0)
df_test_feature['month'] = df_test_feature['pub_date'].apply(lambda x: strptime(x.split()[2], '%b').tm_mon if x 
                                                             != ' noneday' else 0)
df_test_feature['hour'] = df_test_feature['pub_date'].apply(lambda x: strptime(x.split()[4], '%X')[3] if x  != ' noneday' else 0)


In [28]:
df_test_feature.to_csv('./test_feature.csv',index=False,header=True)